[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/fusion_retrieval_with_llamaindex.ipynb)

# 文档搜索中的融合检索

## 概述

此代码实现了一个融合检索系统，该系统结合了基于向量的相似性搜索和基于关键字的 BM25 检索。该方法旨在利用两种方法的优势，以提高文档检索的整体质量和相关性。

## 动机

传统的检索方法通常依赖于语义理解（基于向量）或关键字匹配（BM25）。每种方法都有其优点和缺点。融合检索旨在结合这些方法，以创建一个更强大、更准确的检索系统，能够有效处理更广泛的查询。

## 关键组件

1. PDF 处理和文本分块
2. 使用 FAISS 和 OpenAI 嵌入创建向量存储
3. 创建用于基于关键字检索的 BM25 索引
4. 融合 BM25 和向量搜索结果以实现更好的检索

## 方法详情

### 文档预处理

1. 使用 SentenceSplitter 加载 PDF 并将其拆分为块。
2. 通过将 't' 替换为空格和清除换行符来清理块（可能解决了特定的格式问题）。

### 向量存储创建

1. 使用 OpenAI 嵌入创建文本块的向量表示。
2. 从这些嵌入创建一个 FAISS 向量存储，以进行高效的相似性搜索。

### BM25 索引创建

1. 从用于向量存储的相同文本块创建 BM25 索引。
2. 这允许与基于向量的方法一起进行基于关键字的检索。

### 查询融合检索

创建两个索引后，查询融合检索将它们结合起来以实现混合检索

## 此方法的优点

1. 提高检索质量：通过结合语义和基于关键字的搜索，系统可以捕获概念相似性和精确的关键字匹配。
2. 灵活性：`retriever_weights` 参数允许根据特定的用例或查询类型调整向量和关键字搜索之间的平衡。
3. 鲁棒性：组合方法可以有效处理更广泛的查询，减轻了单个方法的弱点。
4. 可定制性：该系统可以轻松适应使用不同的向量存储或基于关键字的检索方法。

## 结论

融合检索代表了一种强大的文档搜索方法，它结合了语义理解和关键字匹配的优点。通过利用基于向量和 BM25 的检索方法，它为信息检索任务提供了更全面、更灵活的解决方案。这种方法在概念相似性和关键字相关性都很重要的各个领域都有潜在的应用，例如学术研究、法律文档搜索或通用搜索引擎。

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
!pip install faiss-cpu llama-index python-dotenv

In [ ]:
import os
import sys
from dotenv import load_dotenv
from typing import List
from llama_index.core import Settings
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.schema import BaseNode, TransformComponent
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.legacy.retrievers.bm25_retriever import BM25Retriever
from llama_index.core.retrievers import QueryFusionRetriever
import faiss

# Original path append replaced for Colab compatibility
# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Llamaindex global settings for llm and embeddings
EMBED_DIMENSION=512
Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

### Read Docs

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [ ]:
path = "data/"
reader = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = reader.load_data()
print(documents[0])

### Create Vector Store

In [ ]:
# Create FaisVectorStore to store embeddings
fais_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=fais_index)

### Text Cleaner Transformation

In [ ]:
class TextCleaner(TransformComponent):
    """
    Transformation to be used within the ingestion pipeline.
    Cleans clutters from texts.
    """
    def __call__(self, nodes, **kwargs) -> List[BaseNode]:
        
        for node in nodes:
            node.text = node.text.replace('\t', ' ') # Replace tabs with spaces
            node.text = node.text.replace(' \n', ' ') # Replace paragprah seperator with spacaes
            
        return nodes

### Ingestion Pipeline

In [ ]:
# Pipeline instantiation with: 
# node parser, custom transformer, vector store and documents
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        TextCleaner()
    ],
    vector_store=vector_store,
    documents=documents
)

# Run the pipeline to get nodes
nodes = pipeline.run()

## Retrievers

### BM25 Retriever

In [ ]:
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
)

### Vector Retriever

In [ ]:
index = VectorStoreIndex(nodes)
vector_retriever = index.as_retriever(similarity_top_k=2)

### Fusing Both Retrievers

In [ ]:
retriever = QueryFusionRetriever(
    retrievers=[
        vector_retriever,
        bm25_retriever
    ],
    retriever_weights=[
        0.6, # vector retriever weight
        0.4 # BM25 retriever weight
    ],
    num_queries=1, 
    mode='dist_based_score',
    use_async=False
)

About parameters

1. `num_queries`:  Query Fusion Retriever not only combines retrievers but also can genereate multiple questions from a given query. This parameter controls how many total queries will be passed to the retrievers. Therefore setting it to 1 disables query generation and the final retriever only uses the initial query.
2. `mode`: There are 4 options for this parameter. 
   - **reciprocal_rerank**: Applies reciporical ranking. (Since there is no normalization, this method is not suitable for this kind of application. Beacuse different retrirevers will return score scales)
   - **relative_score**: Applies MinMax based on the min and max scores among all the nodes. Then scaled to be between 0 and 1. Finally scores are weighted by the relative retrievers based on `retriever_weights`.  
      ```math
      min\_score = min(scores)
      \\ max\_score = max(scores)
      ```
   - **dist_based_score**:  Only difference from `relative_score` is the MinMax sclaing is based on mean and std of the scores. Scaling and weighting is the same.
      ```math
       min\_score = mean\_score - 3 * std\_dev
      \\ max\_score = mean\_score + 3 * std\_dev
      ```
   - **simple**: This method is simply takes the max score of each chunk.  

### Use Case example

In [ ]:
# Query
query = "What are the impacts of climate change on the environment?"

# Perform fusion retrieval
response = retriever.retrieve(query)

### Print Final Retrieved Nodes with Scores 

In [ ]:
for node in response:
    print(f"Node Score: {node.score:.2}")
    print(f"Node Content: {node.text}")
    print("-"*100)